In [1]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=5526f32574a4e9fc86e0949ba17960980f5b8e6f1890d1ddb55131e5397cd31e
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [22]:
from datetime import datetime

In [3]:
spark = SparkSession.builder.appName("LeetcodeEasySQLProblemsPartOne").getOrCreate()

# PROBLEM 1
https://leetcode.com/problems/combine-two-tables/

# Objective:
Learn to do a Basic Left Join in spark


```
Table: Person

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| personId    | int     |
| lastName    | varchar |
| firstName   | varchar |
+-------------+---------+
personId is the primary key (column with unique values) for this table.
This table contains information about the ID of some persons and their first and last names.


Table: Address

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| addressId   | int     |
| personId    | int     |
| city        | varchar |
| state       | varchar |
+-------------+---------+
addressId is the primary key (column with unique values) for this table.
Each row of this table contains information about the city and state of one person with ID = PersonId.


Write a solution to report the first name, last name, city, and state of each person in the Person table. If the address of a personId is not present in the Address table, report null instead.

Return the result table in any order.
```





In [6]:
person_data = [
    {'personId': 1, 'firstName': 'Allen', 'lastName': 'Wang'},
    {'personId': 2, 'firstName': 'Alice', 'lastName': 'Bob'},
]
person_df = spark.createDataFrame(person_data)

In [7]:
person_df.show()

+---------+--------+--------+
|firstName|lastName|personId|
+---------+--------+--------+
|    Allen|    Wang|       1|
|    Alice|     Bob|       2|
+---------+--------+--------+



In [8]:
address_data = [
    {'addressId':1, 'personId':2,  'city': 'New York City',  'state': 'New York'},
    {'addressId':2, 'personId':3,  'city': 'Leetcode',  'state': 'California'}
]
address_df = spark.createDataFrame(address_data)

In [10]:
person_df.join(address_df, person_df["personId"] == address_df["personID"], "left").select("firstName", "lastName", "city", "state").show()

+---------+--------+-------------+--------+
|firstName|lastName|         city|   state|
+---------+--------+-------------+--------+
|    Allen|    Wang|         NULL|    NULL|
|    Alice|     Bob|New York City|New York|
+---------+--------+-------------+--------+



# Problem 2
https://leetcode.com/problems/employees-earning-more-than-their-managers/

# Objective
Do a self join in spark and filter



```
Table: Employee

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| name        | varchar |
| salary      | int     |
| managerId   | int     |
+-------------+---------+
id is the primary key (column with unique values) for this table.
Each row of this table indicates the ID of an employee, their name, salary, and the ID of their manager.


Write a solution to find the employees who earn more than their managers.
```




In [12]:
employee_data = [
    {'id': 1, 'name': 'Joe', 'salary': 70000, 'managerId': 3},
    {'id': 2, 'name': 'Henry', 'salary': 80000, 'managerId': 4},
    {'id': 3, 'name': 'Sam', 'salary': 60000, 'managerId': None},
    {'id': 4, 'name': 'Max', 'salary': 90000, 'managerId': None}
]

In [13]:
employee_df = spark.createDataFrame(employee_data)
employee_df.show()

+---+---------+-----+------+
| id|managerId| name|salary|
+---+---------+-----+------+
|  1|        3|  Joe| 70000|
|  2|        4|Henry| 80000|
|  3|     NULL|  Sam| 60000|
|  4|     NULL|  Max| 90000|
+---+---------+-----+------+



In [21]:
from pyspark.sql.functions import col
employee_df.alias("emp1").join(employee_df.alias("emp2"), col("emp1.managerId") == col("emp2.id"), "left").filter(col("emp1.salary") > col("emp2.salary")).select(
    col("emp1.name"), col("emp1.salary")).show()

+----+------+
|name|salary|
+----+------+
| Joe| 70000|
+----+------+



# Problem 3

https://leetcode.com/problems/game-play-analysis-ii/

# Objective
Apply a window function(rank) and filter by it


```
Table: Activity

+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| player_id    | int     |
| device_id    | int     |
| event_date   | date    |
| games_played | int     |
+--------------+---------+
(player_id, event_date) is the primary key (combination of columns with unique values) of this table.
This table shows the activity of players of some games.
Each row is a record of a player who logged in and played a number of games (possibly 0)
before logging out on someday using some device.


Write a solution to report the device that is first logged in for
each player

```


In [23]:
activity_data = [
     {'player_id':1, 'device_id':2, 'event_date':datetime(year=2016, month=3, day=1), 'games_played':5},
     {'player_id':1, 'device_id':2, 'event_date':datetime(year=2016, month=5, day=2), 'games_played':6},
     {'player_id':2, 'device_id':3, 'event_date':datetime(year=2017, month=6, day=25), 'games_played':1},
     {'player_id':3, 'device_id':1, 'event_date':datetime(year=2016, month=3, day=2), 'games_played':0},
     {'player_id':3, 'device_id':4, 'event_date':datetime(year=2018, month=7, day=3), 'games_played':5}
]
activity_df = spark.createDataFrame(activity_data)
activity_df.show()

+---------+-------------------+------------+---------+
|device_id|         event_date|games_played|player_id|
+---------+-------------------+------------+---------+
|        2|2016-03-01 00:00:00|           5|        1|
|        2|2016-05-02 00:00:00|           6|        1|
|        3|2017-06-25 00:00:00|           1|        2|
|        1|2016-03-02 00:00:00|           0|        3|
|        4|2018-07-03 00:00:00|           5|        3|
+---------+-------------------+------------+---------+



In [28]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc

In [35]:
window_specs = Window.partitionBy("player_id").orderBy("event_date")
activity_df.withColumn("rnk", rank().over(window_specs)).filter(col("rnk") == 1).select("player_id", "device_id").show()

+---------+---------+
|player_id|device_id|
+---------+---------+
|        1|        2|
|        2|        3|
|        3|        1|
+---------+---------+

